<a href="https://colab.research.google.com/github/Bhavya2910/Mashup/blob/main/Mashup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytube moviepy youtube-search-python pydub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 701.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [2]:
import os
from pytube import YouTube
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
from moviepy.editor import concatenate_videoclips
from pydub import AudioSegment
import logging
from youtubesearchpython import VideosSearch


In [3]:
import smtplib
from email.message import EmailMessage
import zipfile


In [4]:

def download_videos(singer, n):
    videos = []
    query = f"{singer} official songs"

    try:
        video_search = VideosSearch(query, limit=n)
        results = video_search.result()

        for video in results['result']:
            video_title = video["title"]


            if 'PLAYLIST' not in video_title.upper():
                video_url = f'https://www.youtube.com/watch?v={video["id"]}'
                yt_video = YouTube(video_url)
                stream = yt_video.streams.filter(file_extension='mp4').first()
                stream.download()
                videos.append(stream.title)

                if len(videos) > n:
                    break

    except Exception as e:
        print(f"Error: {e}")

    return videos


In [5]:

def convert_to_audio(video_path):
    try:
        if os.path.exists(video_path):
            audio_path = os.path.splitext(video_path)[0] + ".mp3"
            clip = VideoFileClip(video_path)
            clip.audio.write_audiofile(audio_path)
            return audio_path
        else:
            print(f"Error converting video to audio: Video file not found - {video_path}")
            return None
    except Exception as e:
        print(f"Error converting video to audio: {e}")
        return None


In [6]:

def cut_audio(audio_path, y):
    try:
        if os.path.exists(audio_path):
            cut_audio_path = os.path.splitext(audio_path)[0] + "_cut.mp3"
            clip = AudioFileClip(audio_path)
            cut_clip = clip.subclip(0, min(y, clip.duration))
            cut_clip.write_audiofile(cut_audio_path)
            return cut_audio_path
        else:
            print(f"Error cutting audio: Audio file not found - {audio_path}")
            return None
    except Exception as e:
        print(f"Error cutting audio: {e}")
        return None


In [7]:

def merge_audios(audio_files, output_path):
    try:
        audio_segments = [AudioSegment.from_file(audio_file, format="mp3") for audio_file in audio_files if audio_file is not None and os.path.exists(audio_file)]
        if audio_segments:
            final_segment = sum(audio_segments)
            final_segment.export(output_path, format="mp3")
            print("Audio files merged successfully.")
        else:
            print("No valid audio files to merge.")
    except Exception as e:
        print(f"Error merging audio files: {e}")


In [8]:
import os
def send_email_with_zip(zip_file_path, email_id):
    try:
        msg = EmailMessage()
        msg.set_content("Please find the attached zip file.")

        msg['Subject'] = 'Merged Audio Files'
        msg['From'] = 'bhhavyagoyal243@gmail.com'  # Your Gmail email address
        msg['To'] = email_id

        with open(zip_file_path, 'rb') as file:
            file_data = file.read()
            msg.add_attachment(file_data, maintype='application', subtype='zip', filename='merged_files.zip')

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login('bhhavyagoyal243@gmail.com', 'bwni tiwz btjw jzbi')  # Replace with your Gmail email and password
            server.send_message(msg)

        print("Email sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")


In [9]:

def main():
    singer = input("Enter singer name: ")
    n = int(input("Enter the number of videos to download: "))
    y = int(input("Enter the duration of each video (in seconds): "))
    email_id = input("Enter your email address: ")

    videos = download_videos(singer, n)
    audio_files = []

    for video_title in videos:
        video_path = video_title + ".mp4"
        audio_path = convert_to_audio(video_path)

        if audio_path:
            cut_audio_path = cut_audio(audio_path, y)
            if cut_audio_path:
                audio_files.append(cut_audio_path)

    if audio_files:
        output_path = f"{singer}_merged_output.mp3"
        merge_audios(audio_files, output_path)

        # Create a zip file
        zip_file_path = f"{singer}_merged_files.zip"
        with zipfile.ZipFile(zip_file_path, 'w') as zip_file:
            for audio_file in audio_files:
                zip_file.write(audio_file)

        # Send email with the zip file
        send_email_with_zip(zip_file_path, email_id)

        print("Task completed successfully.")
    else:
        logging.warning("No valid audio files to process.")


if __name__ == "__main__":
    main()

Enter singer name: BTS
Enter the number of videos to download: 3
Enter the duration of each video (in seconds): 20
Enter your email address: bhavyagoyal867@gmail.com
MoviePy - Writing audio in Dynamite.mp3


MoviePy - Done.
MoviePy - Writing audio in Dynamite_cut.mp3


MoviePy - Done.
MoviePy - Writing audio in Life Goes On.mp3


MoviePy - Done.
MoviePy - Writing audio in Life Goes On_cut.mp3


MoviePy - Done.
MoviePy - Writing audio in FAKE LOVE.mp3


MoviePy - Done.
MoviePy - Writing audio in FAKE LOVE_cut.mp3


MoviePy - Done.
Audio files merged successfully.
Email sent successfully.
Task completed successfully.
